<a href="https://colab.research.google.com/github/abiz-ewn/Mental-Health-Support-Chatbot/blob/main/Mental_Health_Support_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_groq langchain_core langchain_community

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("Who is King David?")
print(result.content)

In [ ]:
!pip install pypdf

In [ ]:
!pip install chromadb

In [ ]:
!pip install sentence_transformers

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain .vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os # Import the os module

def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

def create_vector_db():
  loader = DirectoryLoader("/content/data/", glob ='*pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db= Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDb created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """You are a compassionate mental health bot. Respond through
  {context}
  User = {question}
  Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ["context", "question"])
  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain


def main():
  print("Initializing Chatbot.............")
  llm = initialize_llm()

  db_path = "/content/chroma_db"

  if not os.path.exists(db_path):
    vector_db = create_vector_db()
  else:
    embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory = db_path, embedding_function = embeddings)

  qa_chain = setup_qa_chain(vector_db, llm)

  while True:
    query = input("\nHuman: ")
    if query.lower() == "exit":
      print("Chatbot: Take Care of Yourself, Goodbye!")
      break
    response = qa_chain.run(query)
    print(f"Chatbot: {response}")

if __name__ == "__main__":
  main()

In [ ]:
!pip install gradio

In [ ]:
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain .vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import gradio as gr

def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

def create_vector_db():
  loader = DirectoryLoader("/content/data/", glob ='*pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db= Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDb created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """You are a compassionate mental health bot. Respond through
  {context}
  User = {question}
  Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ["context", "question"])
  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain

print("Initializing Chatbot.............")
llm = initialize_llm()

db_path = "/content/chroma_db"

if not os.path.exists(db_path):
  vector_db = create_vector_db()
else:
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma(persist_directory = db_path, embedding_function = embeddings)


qa_chain = setup_qa_chain(vector_db, llm)

def chatbot_response(message, history=None):
    history = history or []
    if not message.strip():
      return history
    response = qa_chain.run(message)
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": response})
    return history


with gr.Blocks(theme = 'Respair/Shiki@1.2.1') as app:
  chatbot = gr.ChatInterface(fn = chatbot_response, title = "Mental Health Chatbot", type = "messages")

app.launch()